# PFR trajectory convex hull demonstration
## Package imports

In [ ]:
# artools
import sys
sys.path.append('../../../artools')
import artools
artools = reload(artools)

# anaconda
from ipywidgets import interact

import scipy as sp
from scipy.spatial import ConvexHull

import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
%matplotlib inline

plt.style.use("ggplot")
from mpl_toolkits.mplot3d import Axes3D

## Define kinetics

We use the van de vusse system to demonstrate:
$$
\begin{align}
\mathrm{A}	&\rightarrow\mathrm{B}\rightarrow\mathrm{C} \\
2\mathrm{A}	&\rightarrow\mathrm{D}
\end{align}
$$

In [ ]:
# 2D van de Vusse kinetics
# A -> B -> C
# 2A -> D


def rate_fn(C,t):
    cA = C[0]
    cB = C[1]
    
    #rate constants
    k1 = 1.0
    k2 = 1.0
    k3 = 10.0

    #r = [rA, rB, rD]
    return sp.array([-k1*cA - 2*k3*cA**2,
                    k1*cA - k2*cB])


## Set of achievable concentrations for a single PFR (no mixing)

In [ ]:
# define feed
Cf = sp.array([1.0, 0.0])

pfr_cs, pfr_ts = artools.pfrTrajectory(Cf, rate_fn, 100)

fig = plt.figure(figsize=(6,6))
ax = fig.gca()

ax.plot(pfr_cs[:, 0], pfr_cs[:, 1], "r-")

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 0.12])
ax.set_xlabel("cA (mol/L)")
ax.set_ylabel("cB (mol/L)")

plt.show(fig)

The set of concentrations for just a PFR from the feed point in simply a line in $c_\mathrm{A}-c_\mathrm{B}$ space because we have assumed that mixing is not available (the plant may already be built without a bypass).

## Set of achievable points for a CSTR-PFR arrangement (no mixing)

In [ ]:
# generate CSTR locus from the feed
cstr_cs, cstr_ts = artools.cstrLocus_fast(Cf, rate_fn, 1000, 100)

# now plot PFR trajectories from each CSTR point
fig = plt.figure(figsize=(6,6))
ax = fig.gca()

ax.plot(cstr_cs[:, 0], cstr_cs[:, 1], "bx")

for cstr_ci in cstr_cs:
    # plot PFR trajectory using CSTR concentration as a feed point
    pfr_cs, pfr_ts = artools.pfrTrajectory(cstr_ci, rate_fn, 100)
    ax.plot(pfr_cs[:, 0], pfr_cs[:, 1], "r:")

ax.text(0.2, 0.02, "X achievable point?")
    
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 0.12])
ax.set_xlabel("cA (mol/L)")
ax.set_ylabel("cB (mol/L)")

plt.show(fig)

Since there is no mixing between streams, we cannot plot a filled region anymore, and the set of achievable points are only those that belong to the PFR or CSTR points in the above figure.

If we wanted to operate at say $c_\mathrm{A}$ = 0.2 mol/L and $c_\mathrm{B}$ = 0.02 mol/L, then it would NOT be feasible because it is not contained within the set of achievable concentrations.